In [1]:
import numpy as np
import pandas as pd

In [2]:
input = []
channel = []
count = 0
for line in open('../data/input1.txt').readlines():
    if(line == '\n'):
        input.append(channel)
        channel = []
    else:
        channel.append([int(n) for n in line.split()])

print(np.shape(input))

(32, 112, 112)


In [3]:
stride = 30
count = 0
doubleword = []
output = []
# Channel
for c in range(len(input)):
    # Row
    for row in range(0, len(input[c]), stride):
        # Column
        for column in range(0, len(input[c][row]), stride):
            for y in range((row - 1), (row + stride + 1)):
                if(y >= len(input[c])):
                    break
                for x in range((column - 1), (column + stride + 1)):
                    if((x < 0) or (y < 0)):
                        count += 1
                        doubleword.append(0)
                    elif((x >= len(input[c][row])) or (y >= len(input[c]))):
                        # print(count, x, y)
                        for a in range((count % 8) + 1, 9):
                            doubleword.append(0)
                        count = 8
                    else:
                        count += 1
                        doubleword.append(input[c][y][x])
                    
                    if((count % 8) == 0):
                        count = 0
                        output.append(doubleword)
                        # print(doubleword)
                        doubleword = []
                        if((x >= len(input[c][row]))):
                            break

In [4]:
print(np.shape(output))

(57120, 8)


In [5]:
hexadecimal = open("input1_data.S", 'w')
hexadecimal.write("# Input for Layer1\n")
hexadecimal.write(".section .rodata\n")
hexadecimal.write(".align 3\n")
hexadecimal.write(".global input1\n")
hexadecimal.write("input1:\n")
hexadecimal.write("\t.dword  \\\n")

for row in range(len(output)):
    hexadecimal.write("\t0x")
    for column in range(len(output[row]) - 1, -1, -1):
        hexadecimal.write('{:02x}'.format(output[row][column]))
    if(row != len(output) - 1):
        hexadecimal.write(", \\\n")

hexadecimal.close()